## Cómo crear y usar un kernel de Jupyter a partir de un entorno virtual creado con `uv`

Sigue estos pasos para que tu entorno virtual (creado con `uv`) pueda ser seleccionado y usado como kernel en Jupyter Notebook o Jupyter Lab:

1. **Activa tu entorno virtual**
   ```bash
   source .venv/bin/activate

(Reemplaza .venv por el nombre de tu entorno si es diferente)

2. Instala ipykernel en el entorno
   ```bash
   pip install ipykernel

3. Crea y registra el kernel para Jupyter
   ```bash
   python -m ipykernel install --user --name mlops-udm --display-name "mlops udm"
   
* Pon el nombre que deseas, en este caso usaré mlops-udm pero puedes asignar el que desees. 

* Puedes cambiar mlops-udm y "mlops udm" por el nombre que prefieras para identificar tu kernel.

----> Ve al menú superior: Kernel > Cambiar kernel... y elige "mlops udm".
_______

## Escenario 1: Un cientifico de datos participando de una competición en kaggle

MLflow setup:
* Tracking server: no
* Backend store: sistema de archivos local
* Artifacts store: sistema de archivos local

Usando MLFlow Ui podemos revisar los resultados

In [1]:
import mlflow

En este caso estamos usando el path actual para definir donde queremos guardar las ejecuciones

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns'


Define la ubicación donde se almacenarán los metadatos de los experimentos. En este caso, apunta al directorio mlruns dentro de la carpeta scenarios.

In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/582241962186932225', creation_time=1761773985184, experiment_id='582241962186932225', last_update_time=1761773985184, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/0', creation_time=1761773984653, experiment_id='0', last_update_time=1761773984653, lifecycle_stage='active', name='Default', tags={}>]

### Creemos un experimento y logguemos la corrida

Hagamos una prueba con un dataset de juguete, aca definimos siempre un experimento de mlflow de la siguiente manera:

#### Definición del Experimento

```python 
mlflow.set_experiment("my-experiment-1")
```

Crea o selecciona un experimento llamado "my-experiment-1" donde se registrarán todas las ejecuciones relacionadas.

#### Ejecución del Experimento

```python
with mlflow.start_run():                    
    # Código del experimento
```

El contexto with mlflow.start_run() inicia automáticamente una nueva ejecución y la finaliza al salir del bloque.


## Flujo del Experimento

1. Carga de Datos

- Utiliza el dataset de iris de scikit-learn como ejemplo.

2. Definición de Parámetros: 
- C: Parámetro de regularización para LogisticRegression
- random_state: Semilla para reproducibilidad

- mlflow.log_params() registra estos parámetros para futura referencia

3. Entrenamiento del Modelo 

- Entrena un modelo de regresión logística con los parámetros especificados.

4. Evaluación y Logging de Métricas

- Calcula predicciones en el conjunto de entrenamiento
- Registra la métrica de precisión (accuracy) en MLflow

5. Logging del Modelo

- Guarda el modelo entrenado como un artifact, permitiendo su reutilización posterior.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, f1_score, precision_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run(run_name="logistic_regression"):

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.3, "random_state": 42}
    mlflow.log_params(params)

    mlflow.set_tags({"developer": "Maria Durango",
                    "module": "mlops_tracking",
                    "model": "logistic_regression"})


    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))


    mlflow.sklearn.log_model(lr, name="models", input_example=X[[0]])
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/29 20:57:23 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


default artifacts URI: 'file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/582241962186932225/84115f7332bb49a7a44807ca4a02ce33/artifacts'


In [5]:
# import Random Forest

from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run(run_name="random_forest"):
    # Código del experimento
    params = {"n_estimators": 100, "random_state": 42}

    X, y = load_iris(return_X_y=True)

    mlflow.log_params(params)

    rf = RandomForestClassifier(**params).fit(X, y)
    y_pred = rf.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    mlflow.set_tags({"developer": "Maria Durango",
                    "module": "mlops_tracking",
                    "model": "random_forest"})


    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


default artifacts URI: 'file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/582241962186932225/0bde9bd014994aed8de8b3edab444810/artifacts'


## Exploración de Resultados

### Búsqueda de Experimentos

Lista todos los experimentos disponibles en el tracking server.


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/582241962186932225', creation_time=1761773985184, experiment_id='582241962186932225', last_update_time=1761773985184, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/0', creation_time=1761773984653, experiment_id='0', last_update_time=1761773984653, lifecycle_stage='active', name='Default', tags={}>]

Filtrando por texto: 

In [7]:
mlflow.search_experiments(filter_string="name = 'my-experiment-1'")

[<Experiment: artifact_location='file:///Users/mdurango/Downloads/proyectos/machine-learning-udm/03-tracking/scenarios/mlruns/582241962186932225', creation_time=1761773985184, experiment_id='582241962186932225', last_update_time=1761773985184, lifecycle_stage='active', name='my-experiment-1', tags={}>]

Filtrando por ID

In [8]:
mlflow.search_runs(experiment_ids=['582241962186932225'])

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.n_estimators,params.random_state,params.C,tags.mlflow.runName,tags.model,tags.module,tags.mlflow.source.name,tags.developer,tags.mlflow.source.type,tags.mlflow.user
0,0bde9bd014994aed8de8b3edab444810,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-30 01:51:25.140000+00:00,2025-10-30 01:51:25.208000+00:00,1.000000,100,42,None,random_forest,random_forest,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
1,6f2c2f9c26f44364ba973432f9c32199,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-30 01:51:23.483000+00:00,2025-10-30 01:51:25.020000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
2,ac30c9b686e045e1a36d66033af27654,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:44:19.272000+00:00,2025-10-29 21:44:19.325000+00:00,1.000000,100,42,None,random_forest,random_forest,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
3,be74c0926b6e45aeb4f9cbe59420b898,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:44:15.276000+00:00,2025-10-29 21:44:17.031000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
4,41b8e0a372aa47469e1386bdf670012b,582241962186932225,FAILED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:41:20.215000+00:00,2025-10-29 21:41:20.236000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
5,6ff755e9b7054ad4b81e74ef8aa28b6d,582241962186932225,FAILED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:40:51.731000+00:00,2025-10-29 21:40:51.767000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
6,c5b7cf86bafa41faa7035283b2f35565,582241962186932225,FAILED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:39:45.237000+00:00,2025-10-29 21:39:45.263000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango


In [9]:
mlflow.search_runs(experiment_ids=['582241962186932225']).sort_values(by="metrics.accuracy", ascending=False)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.n_estimators,params.random_state,params.C,tags.mlflow.runName,tags.model,tags.module,tags.mlflow.source.name,tags.developer,tags.mlflow.source.type,tags.mlflow.user
0,0bde9bd014994aed8de8b3edab444810,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-30 01:51:25.140000+00:00,2025-10-30 01:51:25.208000+00:00,1.000000,100,42,None,random_forest,random_forest,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
2,ac30c9b686e045e1a36d66033af27654,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:44:19.272000+00:00,2025-10-29 21:44:19.325000+00:00,1.000000,100,42,None,random_forest,random_forest,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
1,6f2c2f9c26f44364ba973432f9c32199,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-30 01:51:23.483000+00:00,2025-10-30 01:51:25.020000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
3,be74c0926b6e45aeb4f9cbe59420b898,582241962186932225,FINISHED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:44:15.276000+00:00,2025-10-29 21:44:17.031000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
4,41b8e0a372aa47469e1386bdf670012b,582241962186932225,FAILED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:41:20.215000+00:00,2025-10-29 21:41:20.236000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
5,6ff755e9b7054ad4b81e74ef8aa28b6d,582241962186932225,FAILED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:40:51.731000+00:00,2025-10-29 21:40:51.767000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango
6,c5b7cf86bafa41faa7035283b2f35565,582241962186932225,FAILED,file:///Users/mdurango/Downloads/proyectos/mac...,2025-10-29 21:39:45.237000+00:00,2025-10-29 21:39:45.263000+00:00,0.966667,None,42,0.3,logistic_regression,logistic_regression,mlops_tracking,/Users/mdurango/Downloads/proyectos/machine-le...,Maria Durango,LOCAL,mdurango


### Interacción con el model registry

Permite interactuar programáticamente con el tracking server y model registry.

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [11]:
client.search_registered_models()

[]

In [12]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")